In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from skimage.morphology import skeletonize
from skimage import data
import sknw
import cv2
from scipy import ndimage as ndi
from skimage.morphology import watershed
from tqdm import tqdm_notebook

%matplotlib inline

In [ ]:
def my_watershed(energy, markers, mask):
    """
    Watershed wrapper.

    :param energy: Watershed "energy" mask
    :param markers: "kernels" for the watershed algorithm
    :param mask: Binary mask separating a single class
    :return: Image with every instance labeled with a separate number.
    """
    markers = ndi.label(markers, output=np.uint32)[0]
    labels = watershed(255-energy, markers, watershed_line=True, mask=mask)
    return labels

In [ ]:
# semantic = semantic.squeeze()
# semantic[semantic < 10] = 0  # TODO: this removes uncertainty. Might be a better way to do this.
# mask = cv2.equalizeHist(mask.astype('uint8'))
# mask = (mask / 255)*3
# mask = np.exp(mask)
# mask = mask / mask.max()
# mask = (mask * 255).astype('uint8')

# mask = cv2.GaussianBlur(mask.astype('float32'), (15, 15), 0)
# shape = mask.shape[0]
# block_size = shape//4
# if (block_size % 2) != 1:
#     block_size += 1

# high_conf = cv2.adaptiveThreshold(mask.astype('uint8'), 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, -55)

In [ ]:
raster_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/dist_mat_result/rgb_bin_504.tif"

with rasterio.open(raster_path, "r") as src:
    image = src.read(1)
    # image = (image * 255).astype(np.uint8)
    # image = cv2.resize(image, None, fx=0.2, fy=0.2)
    meta = src.meta

In [ ]:
image.dtype

In [ ]:
image[image < 200] = 0
dst = cv2.distanceTransform(new_img, cv2.DIST_L1, 3)

In [ ]:
ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create()
ridges = ridge_filter.getRidgeFilteredImage(image)

In [ ]:
ridges.max()

In [ ]:
graph = sknw.build_sknw(ridges)

In [ ]:
mask = image > 220
markers = image > 220
energy = image.copy()
energy[energy < 150] = 0
labels = my_watershed(energy, markers, mask)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(image, 'gray')

In [ ]:
def make_lines(labels):
    graphs = []       
    for num, label in tqdm_notebook(enumerate(range(1, len(np.unique(labels)) + 1)), total=len(np.unique(labels))):
        label_mask = labels == label
        ske = skeletonize(label_mask).astype(np.uint16)
        graph = sknw.build_sknw(ske)
        graphs.append(graph)
        
#         if num == 100:
#             break
    return graphs

In [ ]:
from skimage.morphology import skeletonize
from skimage import data
import sknw

# open and skeletonize
# img = data.horse()
# ske = skeletonize(~img).astype(np.uint16)
img = labels == 341
ske = skeletonize(img).astype(np.uint16)

# build graph from skeleton
graph = sknw.build_sknw(ske)
plt.figure(figsize=(10, 10))
# draw image
plt.imshow(img, cmap='gray')
print(graph.edges())
# draw edges by pts
for (s,e) in graph.edges():
    ps = graph[s][e]['pts']
    plt.plot(ps[:,1], ps[:,0], 'green')
new_ps = ps.copy()
# draw node by o
node, nodes = graph.node, graph.nodes()
ps = np.array([node[i]['o'] for i in nodes])
plt.plot(ps[:,1], ps[:,0], 'r.')
print(ps)
# title and show
plt.title('Build Graph')
plt.show()

In [ ]:
def calc_dist(input_array):
    shifted_arr = np.roll(input_array, -1, axis=0)
    distance = np.linalg.norm(input_array-shifted_arr,axis=1)[:-1]
    #distance = distance[distance > 1]
    return distance

In [ ]:
graph_lines = make_lines(labels)

In [ ]:
def save_gpd_df(save_path, geom_df, meta=None):
    if meta:
        geom_df.crs = meta.get('crs')
    if os.path.exists(save_path):
        os.remove(save_path)
    geom_df.to_file(save_path, driver='GeoJSON', encoding='utf-8')

In [ ]:
ps

In [ ]:
from shapely.geometry import LineString, Point
import geopandas as gpd
import os

In [ ]:
lines = []
each = 50
for graph in tqdm_notebook(graph_lines, len(lines)):
#     node, nodes = graph.node, graph.nodes()
#     ps = np.array([node[i]['o'] for i in nodes])
    
    node, nodes = graph.node, graph.nodes()
    graph_ps = []
    for (s,e) in graph.edges():
        ps = []
        weight = graph[s][e]['weight']
        if weight < 10:
            continue
        ps.append(nodes[s]['o'])
        pts = graph[s][e]['pts']
        ps.extend(pts)
        ps.append(nodes[e]['o'])
        ps = np.array(ps)
        #print(len(ps))
        if len(ps) > each:
            ps = ps[::each]
        graph_ps.extend(ps)
    graph_ps = np.array(graph_ps)
    if len(graph_ps) < 2:
        continue
    #print(graph_ps.shape)
    #print(np.unique(graph_ps))
    y_axis_var = np.var(graph_ps[:,1])
    x_axis_var = np.var(graph_ps[:,0])
    if y_axis_var > x_axis_var:
    #print(y_axis_var, x_axis_var)
        ind = np.lexsort((graph_ps[:,0],graph_ps[:,1]))
    else:
        ind = np.lexsort((graph_ps[:,1],graph_ps[:,0]))
    all_coords = graph_ps[ind]
    graph_ps = sort_points_recur(all_coords)
    #print(np.unique(graph_ps))
    #print(graph_ps.shape)
    graph_ps =  graph_ps * 1 / 0.2
    # change x and y
    ps_yx = graph_ps.copy()
    ps_yx[:, 1] = graph_ps[:, 0]
    ps_yx[:, 0] = graph_ps[:, 1]
    #
    converted = [meta['transform'] * pt for pt in ps_yx]
#     for pt in converted:
#         lines.append(Point(pt))
    line = LineString(converted)
    lines.append(line)

In [ ]:
def sort_points_recur(graph_ps_x):
    input_shape = graph_ps_x.shape
    graph_index = 0
    sorted_graph_pts = [graph_ps_x[graph_index]]
    
    for i in range(len(graph_ps_x)):
        #point = 
        graph_ps_x = np.delete(graph_ps_x, graph_index, axis=0)
        dists = cdist(np.expand_dims(sorted_graph_pts[i], axis=0), graph_ps_x)
        #print(dists.shape)
        if len(dists[0]) > 0:
            graph_index = np.argmin(dists)
            graph_min_val = np.min(dists)
            sorted_graph_pts.append(graph_ps_x[graph_index])
    sorted_graph_pts = np.array(sorted_graph_pts)
    #print(sorted_graph_pts.shape, input_shape)
    assert input_shape == sorted_graph_pts.shape
    return sorted_graph_pts

In [ ]:
graph_ps_x.shape[0]

In [ ]:
np.delete(graph_ps_x, 0, axis=0).shape

In [ ]:
sorted_graph_pts = [graph_ps_x[0]]
for i in range(len(graph_ps_x)):
    dists = cdist(np.expand_dims(graph_ps_x[i], axis=0), graph_ps_x[i + 1:])
    
    if len(dists[0]) > 0:
        graph_index = np.argmin(dists)
        sorted_graph_pts.append(graph_ps_x[i + 1])
sorted_graph_pts = np.array(sorted_graph_pts)

In [ ]:
sorted_graph_pts = sort_points_recur(graph_ps_x)

In [ ]:
graph_ps_x[1:].shape

In [ ]:
np.expand_dims(graph_ps_x[0], axis=0)

In [ ]:
dists = cdist(sorted_graph_pts, sorted_graph_pts)

In [ ]:
np.argmin(dists)

In [ ]:
np.argpartition(dist_mat, 0)[0]

In [ ]:
np.argmin(dist_mat, axis=0)

In [ ]:
plt.imshow(dists)

In [ ]:
from shapely.ops import linemerge, nearest_points
from shapely.geometry import MultiLineString

In [ ]:
lines = []
each = 50
for num, graph in tqdm_notebook(enumerate(graph_lines), len(lines)):
#     node, nodes = graph.node, graph.nodes()
#     ps = np.array([node[i]['o'] for i in nodes])
    same_row = []
    node, nodes = graph.node, graph.nodes()
    #endpts = np.array([node[i]['o'] for i in nodes])
    #print(endpts)
    for (s,e) in graph.edges():
        ps = []
        weight = graph[s][e]['weight']
        if weight < 10:
            continue
        ps.append(nodes[s]['o'])
        pts = graph[s][e]['pts']
        ps.extend(pts)
        ps.append(nodes[e]['o'])
        ps = np.array(ps)
        #print(len(ps))
        if len(ps) > each:
            ps = ps[::each]

        if len(ps) < 2:
            continue
        ps =  ps * 1 / 0.2
        # change x and y
        ps_yx = ps.copy()
        ps_yx[:, 1] = ps[:, 0]
        ps_yx[:, 0] = ps[:, 1]
        #
        converted = [meta['transform'] * pt for pt in ps_yx]
        #line = LineString(converted)
        same_row.append(line)
    if len(same_row) > 1:
        merged_line = merge_lines(same_row)
#         merged_line = linemerge(same_row)
    elif len(same_row) == 0:
        continue
    else:
        merged_line = line
#     if isinstance(merged_line, MultiLineString):
#         good_line = None
#         min_length = 10000
#         for num, segment in enumerate(merged_line):
#             length = segment.length
#             if length < min_length:
#                 line_to_delete = num
#                 min_length = length
#         #merged_line = good_line
#         fixed_list = list(merged_line).pop(line_to_delete)
#         if isinstance(fixed_list, list):
#             merged_line = merge_lines(fixed_list)
    lines.append(merged_line)

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
graph_ps.shape

In [ ]:
graph_lines[10]

In [ ]:
def dist(a, b):
    """
    Distance between two points
    """
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [ ]:
def merge_lines(multiline):
    merged_line = None
    #print(multiline)
    pt = find_top_left_point(multiline)
    if len(pt) > 1:
        pt = pt[0]
    #print(pt)
    multiline = get_ordered_list(multiline, Point(pt))
#     multiline = sort_lines_recursively(multiline)
    for line_index in range(len(multiline)):
        if not isinstance(merged_line, LineString):
            merged_line = multiline[line_index]
            continue
        
        min_dist = 10000
        for i in [(0, 0), (0, 1), (1, 0), (1, 1)]:
            #print(line_index)
            calc_dist = dist(merged_line.coords[i[0]], multiline[line_index].coords[i[1]])
            if calc_dist < min_dist:
                min_dist = calc_dist
                min_setting = i
        point_to_add = multiline[line_index].coords[-1 * min_setting[1]]
#         point_to_add = multiline[line_index].coords[0]
        old_coords = merged_line.coords[:]
#         print(min_setting)

        if min_setting == (1,1):
            seq_to_merge = multiline[line_index].coords[:]
            seq_to_merge.reverse()
            point_to_add = seq_to_merge[-1 * min_setting[1]]
            old_coords.append(point_to_add)
            new_line = LineString(old_coords)
            line_to_merge = LineString(seq_to_merge)
#         elif min_setting == (0,1):
#             old_coords.reverse()
#             seq_to_merge = multiline[line_index].coords[:]
#             seq_to_merge.reverse()
#             point_to_add = seq_to_merge[-1 * min_setting[1]]
#             old_coords.append(point_to_add)
#             new_line = LineString(old_coords)
#             line_to_merge = LineString(seq_to_merge)
#         elif min_setting == (0,0):
#             old_coords.reverse()
#             seq_to_merge = multiline[line_index].coords[:]
#             point_to_add = seq_to_merge[-1 * min_setting[1]]
#             old_coords.append(point_to_add)
#             new_line = LineString(old_coords)
#             line_to_merge = LineString(seq_to_merge)
        else:
            old_coords.append(point_to_add)
            new_line = LineString(old_coords)
            line_to_merge = multiline[line_index]
        #print(min_setting)
        merged_line = linemerge([new_line, line_to_merge])
    return merged_line

In [ ]:
first_line = None
for x_line in test_line:
    if not isinstance(first_line, LineString):
        first_line = x_line
        continue
    for i in range(len(test_line)):
        if i == 0:
            continue
        pt_1, pt_2 = nearest_points(first_line, test_line[i])
        print(dist(pt_1.coords[0], pt_2.coords[0]))

In [ ]:
graph_lines[340].edges()

In [ ]:
some_graph = graph_lines[62]
test_line = []
node, nodes = some_graph.node, some_graph.nodes()
endpts = np.array([node[i]['o'] for i in nodes])
for (s,e) in some_graph.edges():
    ps = []
    weight = some_graph[s][e]['weight']
    if weight < 10:
        continue
    ps.append(nodes[s]['o'])
    pts = some_graph[s][e]['pts']
    ps.extend(pts)
    ps.append(nodes[e]['o'])
    ps = np.array(ps)
    #print(len(ps))
    if len(ps) > each:
        ps = ps[::each]

    if len(ps) < 2:
        continue
    ps =  ps * 1 / 0.2
    # change x and y
    ps_yx = ps.copy()
    ps_yx[:, 1] = ps[:, 0]
    ps_yx[:, 0] = ps[:, 1]
    #
    converted = [meta['transform'] * pt for pt in ps_yx]
    line = LineString(converted)
    test_line.append(line)

In [ ]:
some_graph.edges()

In [ ]:
test_line

In [ ]:
merged_test = merge_lines(test_line)

In [ ]:
list(merged_test).pop(1)

In [ ]:
multi = []
for line in lines:
    if isinstance(line, MultiLineString):
        multi.append(line)

In [ ]:
simple_lines = []
for line in lines:
    if isinstance(line, LineString):
        simple_lines.append(line)

In [ ]:
len(multi)

In [ ]:
def calc_nearest(first_line, second_line):
    pt_1, pt_2 = nearest_points(first_line, second_line)
    return dist(pt_1.coords[0], pt_2.coords[0])

In [ ]:
def get_ordered_list(lines, line):
    lines.sort(key = lambda p: calc_nearest(p, line))
    return lines

In [ ]:
def sort_lines_recursively(lines):
    new_lines = []
    for line in lines:
        sorted_i = get_ordered_list(lines, line)
        new_lines.append(sorted_i[0])
        lines = lines[1:]
    return new_lines

In [ ]:
def find_top_left_point(list_lines):
    all_coords = []
    for line in list_lines:
        all_coords.extend(line.coords[:])
    all_coords= np.array(all_coords)
    ind = np.lexsort((all_coords[:,1],all_coords[:,0])) 
    all_coords = all_coords[ind]
    return all_coords

In [ ]:
save_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/row_data/504/detected_lines_1.geojson"
line_gdf = gpd.GeoDataFrame(geometry=lines)
save_gpd_df(save_path, line_gdf, meta=meta)

In [ ]:
graph_lines[0].edges()

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(markers, 'gray')

In [ ]:
labels.max()

In [ ]:
plt.imshow(image)